In [256]:
import pandas as pd
import numpy as np
from sklearn import datasets
import plotly as py
import plotly.plotly as plt
import plotly.graph_objs as go
from plotly import __version__
from sklearn import preprocessing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.tools.set_credentials_file(username='j.carpenter_12', api_key='t07tLoWdgxKH9sS8TH1T')


In [357]:
boston = datasets.load_boston()
names = boston.feature_names
target = boston.target

bos = pd.DataFrame(boston.data)
bos.columns = names
bos.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [358]:
trace = go.Scatter(x=bos.RM,
                   y=target,
                   mode ='markers')

In [359]:
plt.iplot([trace])

In [360]:
####Normalize data
val = bos.values
min_max_scaler = preprocessing.MinMaxScaler()
val_scaled = min_max_scaler.fit_transform(val)
bos_norm = pd.DataFrame(val_scaled,columns=bos.columns)
bos_norm.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


In [361]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X= bos_norm
y= target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [262]:
lr = linear_model.LinearRegression()

In [263]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [264]:
lr.score(X_train,y_train)


0.73202104744028751

In [265]:
###plot lr prediction intercept

intercept = lr.intercept_
coef = lr.coef_[0]
x = X_test['RM']
min_x = min(x)
max_x = max(x)

fit = [intercept, coef*max_x + intercept]

df = pd.DataFrame({'Predicted':lr.predict(X_test),'Actual':y_test})


z = np.polyfit(x=df.loc[:,'Predicted'],y=df.loc[:,'Actual'],deg=1)
p = np.poly1d(z)
df['trendline'] = p(df.loc[:,'Predicted'])



trace1 = go.Scatter(x=df['Predicted'],
                   y=df['Actual'],
                   mode ='markers')

trace2 = go.Scatter(
                  x=[df['Predicted'].min(),df['Predicted'].max()] ,
                  y=df['trendline'].sort_values(), 
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit'
                  )

plt.iplot([trace1,trace2])

The Mean Squared Error (MSE) is a measure of how close a fitted line is to data points. For every data point, you take the distance vertically from the point to the corresponding y value on the curve fit (the error), and square the value. Then you add up all those values for all data points, and, in the case of a linear fit, divide by the number of points minus two.** The squaring is done so negative values do not cancel positive values. The smaller the Mean Squared Error, the closer the fit is to the data. The MSE has the units squared of whatever is plotted on the vertical axis.

In [266]:
prediction = lr.predict(X_test)

##calculate the mean squared error of the model
mseFull = np.mean((y_test - lr.predict(X_test))**2)
print(mseFull)

13.7333069614


In [267]:
##calculate the score of our model predicted/actual
lr.score(X_test,y_test)

0.78970128695033259

# Feature Selection  

In machine learning and statistics, feature selection, also known as variable selection, attribute selection or variable subset selection, is the process of selecting a subset of relevant features (variables, predictors) for use in model construction.

The central premise when using a feature selection technique is that the data contains many features that are either redundant or irrelevant, and can thus be removed without incurring much loss of information. Redundant or irrelevant features are two distinct notions, since one relevant feature may be redundant in the presence of another relevant feature with which it is strongly correlated.

In [342]:
from sklearn.feature_selection import SelectFromModel


sfm = SelectFromModel(lr,threshold=0.25)
sfm.fit(X_test,y_test)
print(sfm.fit(X_test,y_test))


SelectFromModel(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
        prefit=False, threshold=0.25)


In [343]:
lr.fit(X_transform,y_test)
score = lr.score(X_transform,y_test)

In [344]:
n_features = sfm.transform(X_test).shape[1]
features = pd.DataFrame({'n_features':n_features,'score':score,'MSE':mseFull},index=[0])
while n_features > 1:
    sfm.threshold += 0.1
    X_transform = sfm.transform(X_test)
    n_features = X_transform.shape[1]
    lr.fit(X_transform,y_test)
    score = lr.score(X_transform,y_test)
    mseFull = np.mean((y_test - lr.predict(X_transform))**2)
    temp = pd.DataFrame({'n_features':n_features,'score':score,'MSE':mseFull},index=[0])
    features = features.append(temp)
    

group_features = features.groupby(features['n_features'],as_index=False).last()
group_features


,MSE,n_features,score
0,25.636478,1,0.607428
1,21.903709,2,0.664588
2,15.159241,3,0.767866
3,14.368996,4,0.779967
4,12.359029,5,0.810746
5,9.312791,6,0.857393
6,9.276388,7,0.857950
7,8.501195,8,0.869821
8,8.124666,9,0.875587
9,8.017986,10,0.877220


From this we can infer that our model performs better the more columns that are included in the linear regression

# Decision Tree Prediction


In [367]:
#As we are predicting a continuous variable, we import the regressor
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state = 100,
                               max_depth=3, min_samples_leaf=5)
dtr.fit(X_train, y_train)


predicted = dtr.predict(X_test)

dtr_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})

dtr_predictions.head()



,Actual,Predicted
0,23.6,22.871831
1,32.4,22.871831
2,13.6,12.894565
3,22.8,22.871831
4,16.1,12.894565


In [369]:
#Model score 
score = dtr.score(X_test,y_test)
print(score)


0.778910370136


From this we can see that our basic Decision Tree outperforms the linear regression

## Using cross validation and grid search to improve models

From the decision tree above we can see that there are many differnt options available to set when creating the model. Going through and adjusting each of these by hand to improve our score would take time. This is where cross validation comes in

In [407]:
from sklearn.model_selection import GridSearchCV

params ={'max_depth':[1,2,3,4,5,6,7,8],
         'min_samples_leaf':[1,2,3,4,5]
         }
grid_search = GridSearchCV(dtr,param_grid=params,cv=5)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()


Best score: 0.8167652691302683
Best parameters: {'max_depth': 6, 'min_samples_leaf': 3}
predicted score against actual: 0.898348


,Actual,Predicted
0,23.6,20.844961
1,32.4,31.291667
2,13.6,16.823810
3,22.8,23.542857
4,16.1,16.823810


From the above we can see that by using grid_search to help tune the model for us it has had a large impact, improving the model's accuracy by over ten percent

# Random Forest Prediction

In this section we will look at whether a Random Forest model prediction will be more accurate than the linear regression model above. 

In [402]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 15, random_state = 100,
                               max_depth=3, min_samples_leaf=5)


rf.fit(X_train, y_train)
predicted = rf.predict(X_test)
rf_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
rf_score = rf.score(X_test,y_test)
print("predicted score against actual: %f" % rf_score)
rf_predictions.head()

predicted score against actual: 0.824989


,Actual,Predicted
0,23.6,23.539519
1,32.4,25.743848
2,13.6,16.055184
3,22.8,23.874663
4,16.1,15.265623


Grid search on random forest to tune model

In [406]:
params ={'n_estimators':[130,140,150],'max_depth':[20,21,22,23,24,25],
         'min_samples_leaf':[1]
         }
grid_search = GridSearchCV(rf,param_grid=params)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()


Best score: 0.8579498759563667
Best parameters: {'max_depth': 23, 'min_samples_leaf': 1, 'n_estimators': 140}
predicted score against actual: 0.886815


,Actual,Predicted
0,23.6,22.313571
1,32.4,30.355000
2,13.6,15.726429
3,22.8,23.316429
4,16.1,16.862857


From the above that although the random forest initially performed better than the decision tree when tuning with grid search the decision tree has created a better model

# Gradient Boosted Regression

In [408]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators= 15, random_state = 100,
                               max_depth=3, min_samples_leaf=5)


gbr.fit(X_train, y_train)
predicted = gbr.predict(X_test)
gbr_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
gbr_score = gbr.score(X_test,y_test)
print("predicted score against actual: %f" % gbr_score)
gbr_predictions.head()


predicted score against actual: 0.828524


,Actual,Predicted
0,23.6,22.897290
1,32.4,27.030809
2,13.6,17.457687
3,22.8,23.381114
4,16.1,17.428375


In [410]:
params ={'n_estimators':[50,100,130],'max_depth':[20,21,22,23],
         'min_samples_leaf':[1,2,3,4,5]
         }
grid_search = GridSearchCV(gbr,param_grid=params)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()

Best score: 0.8489300863222985
Best parameters: {'max_depth': 21, 'min_samples_leaf': 3, 'n_estimators': 130}
predicted score against actual: 0.916756


,Actual,Predicted
0,23.6,23.295662
1,32.4,31.242559
2,13.6,18.104824
3,22.8,23.785077
4,16.1,17.252985
